In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
dfcp = pd.read_csv('Company_Profiler.csv')
df = pd.read_csv('all.csv')

In [0]:
dfcp.head()

,KF_ID,Number of Data Points for Each Company,Number of Years We Have Information,IndustryName,IndustrySegmentName
0,1,8,1,['Financials'],['Consumer Finance']
1,2,14058,7,"['Consumer Services', 'Retail (Services)']","['Specialized Consumer Services', 'unknown']"
2,3,87477,10,"['Consumer Goods', 'Fast Moving Consumer Goods...","['Household and Personal Care', 'unknown']"
3,4,30390,10,"['Retailers (Services)', 'Consumer Services', ...","['unknown', 'Food Retailers and Wholesalers']"
4,5,77,2,['Industrials'],"['Transportation Infrastructure', 'Road and Ra..."


In [0]:
# Analyze positions of a company
def jobs_of_company(dataf,kf_id):
    jb = {}
    for i in range(len(dataf)):
        if dataf['KF_ID'][i] == kf_id:
            if dataf['KF_ID'][i] not in jb.keys():
                aa = {}
                aa[dataf['JobName'][i]] = [dataf['CalendarYear'][i]]
                jb[dataf['KF_ID'][i]] = aa
            else:
                if dataf['JobName'][i] not in jb[dataf['KF_ID'][i]].keys():
                    jb[dataf['KF_ID'][i]][dataf['JobName'][i]] = [dataf['CalendarYear'][i]]
                else:
                    if dataf['CalendarYear'][i] not in jb[dataf['KF_ID'][i]][dataf['JobName'][i]]:
                        jb[dataf['KF_ID'][i]][dataf['JobName'][i]].append(dataf['CalendarYear'][i])
        else:
            continue
    job_list = pd.DataFrame(jb)
    for j in job_list.values:
        j[0].sort()
    return job_list

In [0]:
# Remuneration of Overall Job
def remuneration_of_job(dataf,kf_id, salary_column_name):
    roj = {}
    for i in range(len(dataf)):
        if dataf['KF_ID'][i] == kf_id:
            if dataf['CalendarYear'][i] not in roj.keys():
                roj[dataf['CalendarYear'][i]] = dataf[salary_column_name][i]
            else:
                roj[dataf['CalendarYear'][i]] += dataf[salary_column_name][i]
        else:
            continue
    return roj

In [0]:
kf_id = 3

In [0]:
# Number of records/data points for the company
print(dfcp['Number of Data Points for Each Company'][kf_id-1])

87477


In [0]:
# Number of years for which we have the data
print(dfcp['Number of Years We Have Information'][kf_id-1])

10


In [0]:
# IndustryName for the company
print(dfcp['IndustryName'][kf_id-1])

['Consumer Goods', 'Fast Moving Consumer Goods (Industrial/Mfg.)', 'Diversified Industrials (Industrial/Mfg.)']


In [0]:
# IndustrySegmentName for the company
print(dfcp['IndustrySegmentName'][kf_id-1])

['Household and Personal Care', 'unknown']


In [0]:
# number of unique jobs 
job_list = jobs_of_company(df,kf_id)
print(len(job_list))
job_list.head()

In [0]:
# Explore overall salary trends for the company
salary_dict = remuneration_of_job(df,kf_id, "Total Remuneration")
sorted_salary_dict = sorted(salary_dict.items(), key=lambda e:e[0], reverse=False)
sorted_salary_df = pd.DataFrame(sorted_salary_dict)
sorted_salary_df.rename(columns = {0:"year",1:"Total Remuneration"},inplace = True)
sorted_salary_df.head(10)
calenyear = []
totalremu = []
for i in range(len(sorted_salary_dict)):
    calenyear.append(str(sorted_salary_dict[i][0]))
    totalremu.append(sorted_salary_dict[i][1])
plt.plot(calenyear,totalremu)
plt.xlabel('Years')
plt.ylabel('Total Remuneration')
plt.show()

In [0]:
# 3-b Explore them at a Job level instead of an overall level as well. (maybe a good visualization will help)
# salary on job level
# input parameter salary_column_name is the column that user wants to look at
#   e.g. total annual remuneration, base salary, etc
# output: a dataframe, first column: company id
#                      second column: job name
#                      third column: calendar year
#                      forth column: average salary
def salary_job_by_year(dataf, kf_id, job_name, salary_column_name):
    df = dataf[dataf['KF_ID'] == kf_id][['KF_ID','JobName', salary_column_name, 'CalendarYear']]
    output = df.groupby(['KF_ID','JobName', 'CalendarYear']).mean().reset_index()
    return output.loc[(output['JobName'] == job_name)]

In [0]:
# to plot:
output = salary_job_level(df, kf_id, job_name, salary_column_name)
years = list(df['CalendarYear'])
salary = list(df[salary_column_name])
plt.plot(years,salary)
plt.xlabel('year')
plt.ylabel('average salary')
plt.show()

3-c Explore differences in salary between different levels of employees. Does the CEO earn 10 times a normal worker?

For 3-c, start off by selecting a skill level variable. This can be either ReferenceLevelNum or IncumbemtPointCount.
Make sure that your code allows the user to select either of the skill variable.

Now what we need to check is the salary values associated with different skill level.
One possible option is to split the skill variable into buckets and then calculate the mean/median salary variable for that bucket.

For eg, you take IncumbentPointCount as the skill variable. You see that for the specific company,
the variable varies from 20-120. Then you can probably create 10 buckets of 20-30, 30-40 and so on.
For each bucket, calculate the mean/median of the salary variable.

Here salary variable can be any variable that the user wishes to explore. So let your code be flexible
enough to accept either Total Cash or Total Remuneration or Base Salary.


Next stage * you dont have to do this now *

In the next stage, we try to establish a more a concrete relationship between skill and salary for a company.
One of the ways to do that is to model for salary using skill level.
We ultimately need an equation like:
Salary = a*skill + b*skill^2 + c*skill^3 + intercept

We shall discuss this further in your next task.

In [0]:
# salary on job level
# input parameter: salary_column_name is the column that user wants to look at
#   e.g. total annual remuneration, base salary, etc
# input parameter: if mean is True, return average,
#                   else, return median
# output: a dataframe, first column: company id
#                      second column: job name
#                      third column: calendar year
#                      forth column: average salary
# Note: as for grouping into bins, we don't know what variable user will select,
#       so we do not do grouping here. It should be done once we got the result.
def salary_by_level(dataf, kf_id, level_column_name, salary_column_name, mean = True):
    df = dataf[dataf['KF_ID'] == kf_id][['KF_ID', level_column_name, salary_column_name]]
    if mean:
        output = df.groupby(['KF_ID', level_column_name]).mean().reset_index()
    else:
        output = df.groupby(['KF_ID', level_column_name]).median().reset_index()
    return output.loc[(output['KF_ID'] == kf_id) & (output['JobName'] == job_name)]

    df = dataf[dataf['KF_ID'] == kf_id][['KF_ID','JobName', 'Total Annual Remuneration', 'CalendarYear']]
    output = df.groupby(['KF_ID','JobName', 'CalendarYear']).mean().reset_index()
    return output